In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import xgboost
import math


In [2]:
#loading the data
df = pd.read_csv('https://raw.githubusercontent.com/viveknunia/assignment_intern_repo_vivek_nunia/main/propulsion_plant_decay/propulsion.csv')

In [6]:
df.head()

,Unnamed: 0,Lever position (lp) [ ],Ship speed (v) [knots],Gas Turbine shaft torque (GTT) [kN m],Gas Turbine rate of revolutions (GTn) [rpm],Gas Generator rate of revolutions (GGn) [rpm],Starboard Propeller Torque (Ts) [kN],Port Propeller Torque (Tp) [kN],HP Turbine exit temperature (T48) [C],GT Compressor inlet air temperature (T1) [C],GT Compressor outlet air temperature (T2) [C],HP Turbine exit pressure (P48) [bar],GT Compressor inlet air pressure (P1) [bar],GT Compressor outlet air pressure (P2) [bar],Gas Turbine exhaust gas pressure (Pexh) [bar],Turbine Injecton Control (TIC) [%],Fuel flow (mf) [kg/s],GT Compressor decay state coefficient.,GT Turbine decay state coefficient.
0,0,9.300,27.0,72762.205,3560.393,9753.812,644.806,644.806,1086.583,288.0,780.304,4.523,0.998,22.879,1.050,90.435,1.790,0.973,0.978
1,1,6.175,18.0,29760.552,2306.825,8780.012,246.011,246.011,776.921,288.0,665.511,2.518,0.998,13.438,1.030,34.596,0.685,0.995,0.975
2,2,3.144,9.0,8375.774,1386.739,7051.621,60.318,60.318,589.764,288.0,580.587,1.392,0.998,7.566,1.020,12.477,0.247,0.980,0.978
3,3,3.144,9.0,8377.589,1386.748,7098.469,60.339,60.339,570.651,288.0,576.565,1.390,0.998,7.409,1.021,11.678,0.231,0.984,1.000
4,4,6.175,18.0,29761.091,2306.825,8782.024,246.021,246.021,769.855,288.0,663.682,2.518,0.998,13.374,1.031,34.154,0.676,0.998,0.980


In [12]:
# model to check the best hyperparameters for xgboost model
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [13]:
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [14]:
regressor = xgboost.XGBRegressor()

In [15]:
random_cv_compressor = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)
            
random_cv_turbine = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [7]:
#splitting the train and test data
df_train = df.iloc[:math.ceil(len(df)*0.8),:]
df_test = df.iloc[math.ceil(len(df)*0.8):,:]
y_train_compressor = df_train[['GT Compressor decay state coefficient.']]
y_test_compressor = df_test[['GT Compressor decay state coefficient.']]
y_train_turbine = df_train[['GT Turbine decay state coefficient.']]
y_test_turbine = df_test[['GT Turbine decay state coefficient.']]
df_train = df_train.drop(['GT Compressor decay state coefficient.','GT Turbine decay state coefficient.'],axis = 1)
df_test = df_test.drop(['GT Compressor decay state coefficient.','GT Turbine decay state coefficient.'],axis = 1)
df_train

,Unnamed: 0,Lever position (lp) [ ],Ship speed (v) [knots],Gas Turbine shaft torque (GTT) [kN m],Gas Turbine rate of revolutions (GTn) [rpm],Gas Generator rate of revolutions (GGn) [rpm],Starboard Propeller Torque (Ts) [kN],Port Propeller Torque (Tp) [kN],HP Turbine exit temperature (T48) [C],GT Compressor inlet air temperature (T1) [C],GT Compressor outlet air temperature (T2) [C],HP Turbine exit pressure (P48) [bar],GT Compressor inlet air pressure (P1) [bar],GT Compressor outlet air pressure (P2) [bar],Gas Turbine exhaust gas pressure (Pexh) [bar],Turbine Injecton Control (TIC) [%],Fuel flow (mf) [kg/s]
0,0,9.300,27.0,72762.205,3560.393,9753.812,644.806,644.806,1086.583,288.0,780.304,4.523,0.998,22.879,1.050,90.435,1.790
1,1,6.175,18.0,29760.552,2306.825,8780.012,246.011,246.011,776.921,288.0,665.511,2.518,0.998,13.438,1.030,34.596,0.685
2,2,3.144,9.0,8375.774,1386.739,7051.621,60.318,60.318,589.764,288.0,580.587,1.392,0.998,7.566,1.020,12.477,0.247
3,3,3.144,9.0,8377.589,1386.748,7098.469,60.339,60.339,570.651,288.0,576.565,1.390,0.998,7.409,1.021,11.678,0.231
4,4,6.175,18.0,29761.091,2306.825,8782.024,246.021,246.021,769.855,288.0,663.682,2.518,0.998,13.374,1.031,34.154,0.676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,5.140,15.0,21626.849,1924.332,8472.960,175.219,175.219,693.097,288.0,632.533,2.086,0.998,11.053,1.027,24.416,0.483
7996,7996,1.138,3.0,651.129,1346.818,6641.606,7.373,7.373,472.445,288.0,549.286,1.111,0.998,6.029,1.019,10.674,0.097
7997,7997,1.138,3.0,4821.812,1382.532,6640.815,9.411,9.411,584.791,288.0,565.619,1.266,0.998,6.884,1.019,4.630,0.227
7998,7998,3.144,9.0,8377.779,1386.750,7099.802,60.339,60.339,574.702,288.0,577.834,1.390,0.998,7.428,1.021,11.843,0.234


In [17]:
#data to numpy
df_train = df_train.to_numpy()
df_test = df_test.to_numpy()

In [1]:
#training for optimal hyperparameters
random_cv_turbine.fit(df_train,y_train_compressor.to_numpy())

In [2]:
random_cv_compressor.fit(df_train,y_train_compressor.to_numpy())

In [20]:
print(random_cv_compressor.best_estimator_)
print(random_cv_turbine.best_estimator_)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=15,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=15,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, random_state=0,
          

In [27]:
regressor_compressor = xgboost.XGBRegressor(base_score=1, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.15, max_delta_step=0, max_depth=5,
             min_child_weight=2, missing=math.nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

regressor_turbine = xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=15,
             min_child_weight=4, missing=math.nan, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) 

In [28]:
#trainging with optimal hyperparameters
regressor_compressor.fit(df_train,y_train_compressor.to_numpy())
regressor_turbine.fit(df_train,y_train_turbine.to_numpy())

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=15,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
y_pred_compressor = regressor_compressor.predict(df_test) 
y_pred_turbine = regressor_turbine.predict(df_test) 

In [9]:
#saving pickel file
import pickle
filename= 'finalized_model_xgbregressor_compressor.pkl'
pickle.dump(regressor_compressor,open(filename,'wb'))

filename= 'finalized_model_xgbregressor_turbine.pkl'
pickle.dump(regressor_turbine,open(filename,'wb'))

In [24]:
y_test_compressor.to_numpy()
y_test_turbine.to_numpy()

array([[0.983],
       [0.995],
       [0.98 ],
       ...,
       [0.996],
       [0.975],
       [0.977]])

In [25]:
mean_squared_error(y_pred_compressor,y_test_compressor.to_numpy())

2.863398108391966e-06

In [26]:
mean_squared_error(y_pred_turbine,y_test_turbine.to_numpy())

7.144804340992235e-07

In [19]:
p = df['GT Compressor decay state coefficient.']
p = p.iloc[math.ceil(len(df)*0.8):]
p1 = df['GT Compressor decay state coefficient.']
p1 = p1.iloc[math.ceil(len(df)*0.8):]
p = list(p)
p1 = list(p1)
test1_compressor = pd.DataFrame(p,columns = ['original_compressor_decay_coefficient'])
test1_compressor['predicted_compressor_decay_coefficient'] = y_pred_compressor

test1_turbine = pd.DataFrame(p1,columns = ['original_turbine_decay_coefficient'])
test1_turbine['predicted_turbine_decay_coefficient'] = y_pred_turbine
'''test1_compressor.to_csv('C:/Users/91707/OneDrive - LNMIIT/plants_decay_eval/compressor_prediction.csv')
test1_turbine.to_csv('C:/Users/91707/OneDrive - LNMIIT/plants_decay_eval/turbine_prediction.csv')'''